# **Avaliação 03 - ENGA74 2023.1**
**Aluno:** André Paiva Conrado Rodrigues

## **Importação de dependências para os códigos**

In [1]:
from time import time
import numpy as np
from numpy.random import rand
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## **Funções de ***benchmark*****

Esfera:

In [2]:
#Função Esfera
def esfera(x_array, derivada=False):
    if(not derivada):
        return np.sum(np.power(x_array, 2))
    else:
        return 2 * x_array

Ackley:

In [3]:
#Função Ackley
def ackley(x_array, derivada=False):
    eps_stability = 1e-8
    exp1 = np.exp(-0.2 * np.sqrt(np.sum(np.power(x_array, 2))/x_array.shape[0]))
    exp2 = np.exp(np.sum(np.cos(2 * np.pi * x_array))/x_array.shape[0])
    
    if(not derivada):
        return (-20 * exp1) - exp2 + 20 + np.e
    else:
        aux1 = np.tile(np.sqrt(np.sum(np.power(x_array, 2)) + eps_stability), (x_array.shape[0], 1)).transpose()
        aux2 = np.divide(x_array, aux1)
        exp1 = np.tile(exp1, (x_array.shape[0], 1)).transpose()
        exp2 = np.tile(exp2, (x_array.shape[0], 1)).transpose()
        coef1 = 2.828
        coef2 = np.pi
        if(x_array.shape[0] == 3):         
            coef1 = 5.6562/np.sqrt(3)
            coef2 = 2 * np.pi / 3
        return (coef1 * aux2 * exp1) + (coef2 * exp2 * np.sin(2 * np.pi * x_array))

## **Funções de treinamento**

Gradiente Descendente:

In [4]:
def gradiente(x_array, funcao, alpha, iteracoes=30):
    x_trained_array = np.copy(x_array)
    func_evolution = [funcao(x_array)]
    time_init = time()
    for _ in range(iteracoes):
        deriv = funcao(x_trained_array, derivada=True)
        x_trained_array = x_trained_array - (alpha * deriv)
        func_evolution.append(funcao(x_trained_array))
    func_evolution = np.array([func_evolution])
    elapsed_time = time() - time_init
    return func_evolution, elapsed_time

In [5]:
def treina_gradiente(alpha, dim, funcao='esfera', iteracoes=30):
    init = (np.random.rand(20, dim) * 60) - 30

    evol_train = np.empty((0, iteracoes + 1))
    times = np.array([])

    if(funcao == 'ackley'):
        f = ackley
    else:
        f = esfera

    for i in range(20):
        func_evolution, elapsed_time = gradiente(init[i], f, alpha, iteracoes)
        evol_train = np.append(evol_train, func_evolution, axis=0)
        times = np.append(times, elapsed_time)

    evol_train=pd.DataFrame(np.transpose(evol_train))
    mean_training_time = np.mean(times) * 1000

    fig = px.line(evol_train, title=f"Treino da func. {funcao} em R{dim} com alpha {alpha} - Tempo médio para {iteracoes} iterações: {mean_training_time:.3f} ms")
    fig.update_xaxes(title_text='Iterações')
    fig.update_yaxes(title_text=f'Valor da função {funcao}')
    fig.update_layout(legend_title_text='Inicialização')
    fig.show()

Algoritmo Genético:

In [25]:
def genetico(x_matrix, funcao, iteracoes=30, p_recomb=0.1, p_mutacao = 0.05):
    if(x_matrix.shape[1] == 2):
        col = ['x', 'y', 'fitness', 'iter']
    else:
        col = ['x', 'y', 'z', 'fitness', 'iter']
    df_evolution = pd.DataFrame(columns=col)
    x_pop = np.copy(x_matrix)
    for i in range(iteracoes):
        # Cálculo de Fitness e inserção no dataframe
        x_fitness = np.array([-funcao(candidato) for candidato in x_pop]).reshape(-1, 1)
        iter_atual = np.tile([i], x_matrix.shape[0]).reshape(-1, 1)
        df_data = np.concatenate((x_pop, x_fitness, iter_atual), axis=1)
        df_evolution = pd.concat([df_evolution, pd.DataFrame(df_data, columns=col)]).reset_index(drop=True)
        # Sorting Crescente
        x_sort = np.argsort(x_fitness, axis=0).reshape(-1)
        x_fitness = x_fitness[x_sort]
        x_pop = x_pop[x_sort]
        # Cálculo de probabilidades de seleção
        prob_num = np.array([np.sum(np.arange(1, i+1)) for i in range(1, x_matrix.shape[0]+1)])
        prob_den = np.sum(np.arange(1, x_fitness.shape[0]+1))
        prob = prob_num/prob_den
        # Seleção de elementos
        selecao_prob = np.random.rand(x_fitness.shape[0], 2)
        index_selecao_prob = np.searchsorted(prob, selecao_prob, side='right')
        # Recombinação
        
        




    return df_evolution

    

In [26]:
init = (np.random.rand(20, 2) * 60) - 30
    
df_evol = genetico(init, esfera)

fig = px.scatter(df_evol, x='x', y='y', color='fitness', animation_frame='iter',
                 range_y=[-30, 30], range_x=[-30, 30])
fig.show()

[0.0047619  0.01428571 0.02857143 0.04761905 0.07142857 0.1
 0.13333333 0.17142857 0.21428571 0.26190476 0.31428571 0.37142857
 0.43333333 0.5        0.57142857 0.64761905 0.72857143 0.81428571
 0.9047619  1.        ] [[0.04090729 0.10244583]
 [0.16545699 0.05151677]
 [0.08838707 0.76683137]
 [0.42234228 0.86494644]
 [0.93932218 0.77465616]
 [0.20011372 0.88814675]
 [0.70119031 0.83374799]
 [0.78172834 0.57981986]
 [0.97150237 0.3298474 ]
 [0.2960078  0.22040989]
 [0.87517667 0.37937889]
 [0.38536668 0.41360724]
 [0.19578255 0.63704835]
 [0.96790015 0.74110504]
 [0.52571717 0.58212946]
 [0.80818482 0.87875212]
 [0.88388274 0.79863292]
 [0.87513138 0.5675101 ]
 [0.10340897 0.6264879 ]
 [0.90162634 0.8777519 ]] [[ 3  6]
 [ 7  4]
 [ 5 17]
 [12 18]
 [19 17]
 [ 8 18]
 [16 18]
 [17 15]
 [19 11]
 [10  9]
 [18 12]
 [12 12]
 [ 8 15]
 [19 17]
 [14 15]
 [17 18]
 [18 17]
 [18 14]
 [ 6 15]
 [18 18]]
[0.0047619  0.01428571 0.02857143 0.04761905 0.07142857 0.1
 0.13333333 0.17142857 0.21428571 0.2619

Enxame de Partículas:

## **Exercício 01 - Gradiente**

### 1. Otimização da esfera em $\mathbb{R}^{2}$ com 20 inicializações:

Caso 1.1: $\alpha$ = 0.1, treinamento por 30 iterações

In [8]:
treina_gradiente(1e-1, 2, 'esfera', 30)

Caso 1.2: $\alpha$ = 0.01, treinamento por 30 iterações

In [9]:
treina_gradiente(1e-2, 2, 'esfera', 30)

Caso 1.3: $\alpha$ = 10, treinamento por 30 iterações

In [10]:
treina_gradiente(1e1, 2, 'esfera', 30)

### 2. Otimização da esfera em $\mathbb{R}^{3}$ com 20 inicializações:

Caso 2.1: $\alpha$ = 0.1, treinamento por 30 iterações

In [11]:
treina_gradiente(1e-1, 3, 'esfera', 30)

Caso 2.2: $\alpha$ = 0.01, treinamento por 30 iterações

In [12]:
treina_gradiente(1e-2, 3, 'esfera', 30)

Caso 2.3: $\alpha$ = 10, treinamento por 30 iterações

In [13]:
treina_gradiente(1e1, 3, 'esfera', 30)

### 3. Otimização da função Ackley em $\mathbb{R}^{2}$ com 20 inicializações:

Caso 3.1: $\alpha$ = 0.01, treinamento por 30 iterações

In [14]:
treina_gradiente(1e-2, 2, 'ackley', 30)

Caso 3.2: $\alpha$ = 0.1, treinamento por 30 iterações

In [15]:
treina_gradiente(1e-1, 2, 'ackley', 30)

### 4. Otimização da função Ackley em $\mathbb{R}^{3}$ com 20 inicializações:

Caso 4.1: $\alpha$ = 0.005, treinamento por 30 iterações

In [16]:
treina_gradiente(5e-3, 3, 'ackley', 30)

Caso 4.2: $\alpha$ = 0.01, treinamento por 30 iterações

In [17]:
treina_gradiente(1e-2, 3, 'ackley', 30)

## **Exercício 02 - Algoritmo Genético**

## **Exercício 03 - Enxame de Partículas**

## **Exercício 04 - Considerações**